# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
#turn off warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
humdisplay = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                        geo = True,
                                        tiles = "OSM",
                                        color="Humidity")
# Display the map
# YOUR CODE HERE
humdisplay

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df
# A max temperature lower than 27 degrees but higher than 21
ideal_weather = ideal_weather.loc[(ideal_weather["Max Temp"]>21 ) & (ideal_weather["Max Temp"]<27 )] 
# Wind speed less than 4.5 m/s
ideal_weather = ideal_weather.loc[(ideal_weather["Wind Speed"]<4.5 )]
# Zero cloudiness
ideal_weather = ideal_weather.loc[(ideal_weather["Cloudiness"]==0) ]                               
# Drop any rows with null values
# Use this website for help https://www.w3schools.com/python/pandas/ref_df_dropna.asp
ideal_weather.dropna()
# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR Code 
hotel_df = pd.DataFrame(ideal_weather,columns=["City","Country","Lng","Lat","Humidity"])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE for the blank https://www.geeksforgeeks.org/how-to-add-empty-column-to-dataframe-in-pandas/
hotel_df["Hotel Name"]=""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
45,kapaa,US,-159.3190,22.0752,84,
51,hilo,US,-155.0900,19.7297,83,
63,banda,IN,80.3333,25.4833,52,
81,makakilo city,US,-158.0858,21.3469,81,
152,kahului,US,-156.4700,20.8947,60,
197,gat,IL,34.7642,31.6100,100,
211,laguna,US,-121.4238,38.4210,79,
240,tikaitnagar,IN,81.5833,26.9500,59,
265,san quintin,MX,-115.9500,30.4833,74,
340,santa rosalia,MX,-112.2833,27.3167,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 100000 
params = {
    "format":"json"
    "categories":"Hotels",
    "apiKey": geoapify_key,
    "limit":20
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (1978477105.py, line 5)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
Hotdisplay = hotel_df.hvplot.points("Lng",
                                     "Lat",
                                        geo = True,
                                        tiles = "OSM",
                                        color="Hotel Name")
# Display the map
# YOUR CODE HERE
Hotdisplay

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)